### Passo 1 - Importar Arquivos e Bibliotecas

#### Importar Bibliotecas

In [2]:
import pandas as pd
import win32com.client as win32
import pathlib

#### Importar Bases de Dados

In [3]:
emails = pd.read_excel(r'Bases de Dados/Emails.xlsx')
lojas = pd.read_csv(r'Bases de Dados/Lojas.csv', encoding='latin', sep=';')
vendas = pd.read_excel(r'Bases de Dados/Vendas.xlsx')

#### Visualizar Bases e Informações

In [4]:
display(emails)
display(lojas)
display(vendas)

,Loja,Gerente,E-mail
0,Iguatemi Esplanada,Helena,rafa.r.oliveira.rrdo+helena@gmail.com
1,Shopping Midway Mall,Alice,rafa.r.oliveira.rrdo+alice@gmail.com
2,Norte Shopping,Laura,rafa.r.oliveira.rrdo+laura@gmail.com
3,Shopping Iguatemi Fortaleza,Manuela,rafa.r.oliveira.rrdo+manuela@gmail.com
4,Shopping União de Osasco,Valentina,rafa.r.oliveira.rrdo+valentina@gmail.com
5,Shopping Center Interlagos,Sophia,rafa.r.oliveira.rrdo+sophia@gmail.com
6,Rio Mar Recife,Isabella,rafa.r.oliveira.rrdo+isabella@gmail.com
7,Salvador Shopping,Heloisa,rafa.r.oliveira.rrdo+heloisa@gmail.com
8,Rio Mar Shopping Fortaleza,Luiza,rafa.r.oliveira.rrdo+luiza@gmail.com
9,Shopping Center Leste Aricanduva,Julia,rafa.r.oliveira.rrdo+julia@gmail.com


,ID Loja,Loja
0,1,Iguatemi Esplanada
1,2,Shopping Midway Mall
2,3,Norte Shopping
3,4,Shopping Iguatemi Fortaleza
4,5,Shopping União de Osasco
5,6,Shopping Center Interlagos
6,7,Rio Mar Recife
7,8,Salvador Shopping
8,9,Rio Mar Shopping Fortaleza
9,10,Shopping Center Leste Aricanduva


,Código Venda,Data,ID Loja,Produto,Quantidade,Valor Unitário,Valor Final
0,1,2019-01-01,1,Sapato Estampa,1,358,358
1,1,2019-01-01,1,Camiseta,2,180,360
2,1,2019-01-01,1,Sapato Xadrez,1,368,368
3,2,2019-01-02,3,Relógio,3,200,600
4,2,2019-01-02,3,Chinelo Liso,1,71,71
...,...,...,...,...,...,...,...
100994,69996,2019-12-26,17,Short Listrado,2,102,204
100995,69996,2019-12-26,17,Mochila,4,270,1080
100996,69996,2019-12-26,17,Pulseira Estampa,1,87,87
100997,69997,2019-12-26,11,Camisa Listrado,1,108,108


In [5]:
emails.info()
print( 50 * '-')
lojas.info()
print( 50 * '-')
vendas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26 entries, 0 to 25
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Loja     26 non-null     object
 1   Gerente  26 non-null     object
 2   E-mail   26 non-null     object
dtypes: object(3)
memory usage: 756.0+ bytes
--------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25 entries, 0 to 24
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   ID Loja  25 non-null     int64 
 1   Loja     25 non-null     object
dtypes: int64(1), object(1)
memory usage: 532.0+ bytes
--------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100999 entries, 0 to 100998
Data columns (total 7 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   Código Venda    100999 non-null  int64  

### Passo 2 - Definir Criar uma Tabela para cada Loja e Definir o dia do Indicador

### Tratamento de Dados

##### Incluir o nome da loja no DataFrame Vendas

In [6]:
vendas = vendas.merge(lojas, on='ID Loja')
display(vendas)

,Código Venda,Data,ID Loja,Produto,Quantidade,Valor Unitário,Valor Final,Loja
0,1,2019-01-01,1,Sapato Estampa,1,358,358,Iguatemi Esplanada
1,1,2019-01-01,1,Camiseta,2,180,360,Iguatemi Esplanada
2,1,2019-01-01,1,Sapato Xadrez,1,368,368,Iguatemi Esplanada
3,21,2019-01-02,1,Camisa Gola V Listrado,2,116,232,Iguatemi Esplanada
4,34,2019-01-02,1,Sapato Listrado,1,363,363,Iguatemi Esplanada
...,...,...,...,...,...,...,...,...
100994,69846,2019-12-25,8,Short Estampa,2,96,192,Salvador Shopping
100995,69846,2019-12-25,8,Tênis Estampa,5,256,1280,Salvador Shopping
100996,69850,2019-12-25,8,Calça Estampa,4,177,708,Salvador Shopping
100997,69972,2019-12-26,8,Terno Liso,3,720,2160,Salvador Shopping


#### Filtrar as Informações de Vendas por Loja

In [7]:
   dicionario_lojas = {}

   for loja in lojas['Loja']:
    dicionario_lojas[loja] =  vendas.loc[vendas['Loja']== loja, :]

In [8]:
display(dicionario_lojas['Iguatemi Esplanada'])

,Código Venda,Data,ID Loja,Produto,Quantidade,Valor Unitário,Valor Final,Loja
0,1,2019-01-01,1,Sapato Estampa,1,358,358,Iguatemi Esplanada
1,1,2019-01-01,1,Camiseta,2,180,360,Iguatemi Esplanada
2,1,2019-01-01,1,Sapato Xadrez,1,368,368,Iguatemi Esplanada
3,21,2019-01-02,1,Camisa Gola V Listrado,2,116,232,Iguatemi Esplanada
4,34,2019-01-02,1,Sapato Listrado,1,363,363,Iguatemi Esplanada
...,...,...,...,...,...,...,...,...
3975,69899,2019-12-25,1,Sapato Xadrez,4,368,1472,Iguatemi Esplanada
3976,69903,2019-12-25,1,Terno Liso,5,720,3600,Iguatemi Esplanada
3977,69928,2019-12-25,1,Meia,1,30,30,Iguatemi Esplanada
3978,69944,2019-12-25,1,Gorro Xadrez,5,85,425,Iguatemi Esplanada


#### Definindo o Dia Indicador

In [9]:
dia_indicador = vendas['Data'].max()

In [10]:
print(dia_indicador)
print('{}/{}'.format(dia_indicador.day,dia_indicador.month))

2019-12-26 00:00:00
26/12


### Passo 3 - Salvar a planilha na pasta de backup

#### Criando o Caminho para a Pasta Backup

In [11]:
caminho_backup = pathlib.Path(r"Backup Arquivos Lojas")

#### Criar Pasta para Loja caso não Haja, e Salvar a Planilha da Loja na Pasta Correspondente

In [12]:
arquivos_pasta_backup = caminho_backup.iterdir()
lista_nomes_backup = [arquivo.name for arquivo in arquivos_pasta_backup]

for loja in dicionario_lojas:
    if loja not in lista_nomes_backup:
        nova_pasta = caminho_backup / loja
        nova_pasta.mkdir()

    nome_arquivo = '{}_{}_{}.xlsx'.format(dia_indicador.day,dia_indicador.month, loja)
    local_arquivo = caminho_backup / loja / nome_arquivo
    dicionario_lojas[loja].to_excel(local_arquivo, index=False)

### Passo 4 - Calcular o Indicador e Automatizar para Todas as Lojas

#### Definição de Metas

In [13]:
meta_faturamento_dia = 1000
meta_faturamento_ano = 1650000
meta_qtdeprodutos_dia = 4
meta_qtdeprodutos_ano = 120
meta_ticketmedia_dia = 500
meta_ticketmedia_ano = 500


#### Executar Cálculos de cada Loja e Enviar por e-mail para o gerente Automatizando para todas as lojas

In [99]:
for loja in dicionario_lojas:

    vendas_loja = dicionario_lojas[loja]
    vendas_loja_dia = vendas_loja.loc[vendas_loja['Data']== dia_indicador, :]

    #faturamento ano
    faturamento_ano = vendas_loja['Valor Final'].sum()
    #faturamento dia
    faturamento_dia = vendas_loja_dia['Valor Final'].sum()

    #diversidade de produtos
    qtde_produtos_ano = len(vendas_loja['Produto'].unique())
    qtde_produtos_dia = len(vendas_loja_dia['Produto'].unique())

    #ticket médio
    valor_venda = vendas_loja[['Código Venda','Quantidade','Valor Final']].groupby('Código Venda').sum()
    ticket_medio_ano = valor_venda['Valor Final'].mean()
    valor_venda_dia =  vendas_loja_dia[['Código Venda','Quantidade','Valor Final']].groupby('Código Venda').sum()
    ticket_medio_dia = valor_venda_dia['Valor Final'].mean()

    #enviar e-mail
    outlook = win32.Dispatch('outlook.application')

    nome = emails.loc[emails['Loja']== loja, 'Gerente'].values[0]
    mail = outlook.CreateItem(0)
    mail.To = emails.loc[emails['Loja']== loja, 'E-mail'].values[0]
    mail.Subject = f'OnePage Dia {dia_indicador.day}/{dia_indicador.month} - Loja {loja}'

    if faturamento_dia >= meta_faturamento_dia:
        cor_fat_dia = '#56fc03'
    else:
        cor_fat_dia = '#fc0320'

    if faturamento_ano >= meta_faturamento_ano:
        cor_fat_ano = '#56fc03'
    else:
        cor_fat_ano = '#fc0320'

    if qtde_produtos_dia >= meta_qtdeprodutos_dia:
        cor_qtde_dia = '#56fc03'
    else:
        cor_qtde_dia = '#fc0320'

    if qtde_produtos_ano >= meta_qtdeprodutos_ano:
        cor_qtde_ano = '#56fc03'
    else:
        cor_qtde_ano = '#fc0320'

    if ticket_medio_dia >= meta_ticketmedia_dia:
        cor_ticket_dia = '#56fc03'
    else:
        cor_ticket_dia = '#fc0320'

    if ticket_medio_ano >= meta_ticketmedia_ano:
        cor_ticket_ano = '#56fc03'
    else:
        cor_ticket_ano = '#fc0320'
    
    
    
    mail.HTMLBody = f'''
    
    <p>Bom dia {nome}, </p> 

    <p>Apresentado a seguir OnePage com o resultado da <strong>Loja {loja}</strong> no <strong>dia {dia_indicador.day}/{dia_indicador.month}</strong>:</p>

    <h2>ONEPAGE DIA</h2>
    <table font-family: poppis border="0.5" width="100%">
        <tr>
            <th>Indicador</th>
            <th>Meta</th>
            <th>Resultado</th>
            <th>Cenário Dia</th>
        </tr>
        <tr>
            <td>Faturamento</td>
            <td style="text-align: center">R$ {meta_faturamento_dia:.2f}</td>
            <td style="text-align: center">R$ {faturamento_dia:.2f}</td>  
            <td style="text-align: center"><font color="{cor_fat_dia}">◙</font></td>
        </tr>    
        <tr>
            <td>Diversidade de Produtos</td>
            <td style="text-align: center">{meta_qtdeprodutos_dia}</td>
            <td style="text-align: center">{qtde_produtos_dia}</td>
            <td style="text-align: center"><font color="{cor_qtde_dia}">◙</font></td>
        </tr>
        <tr>
            <td>Ticket Médio</td>
            <td style="text-align: center">R$ {meta_ticketmedia_dia:.2f}</td> 
            <td style="text-align: center">R$ {ticket_medio_dia:.2f}</td>
            <td style="text-align: center"><font color="{cor_ticket_dia}">◙</font></td>
        </tr>
    </table>
    <br>
    <h2>ONEPAGE ANO</h2>
    <table font-family: poppis, border="0.5" width="100%">
        <tr>
            <th>Indicador</th>
            <th>Meta</th>
            <th>Resultado</th>
            <th>Cenário Ano</th>
        </tr>
        <tr>
            <td>Faturamento</td>
            <td style="text-align: right">R$ {meta_faturamento_ano:.2f}</td>
            <td style="text-align: right">R$ {faturamento_ano:.2f}</td>  
            <td style="text-align: center"><font color="{cor_fat_ano}">◙</font></td>
        </tr>    
        <tr>
            <td>Diversidade de Produtos</td>
            <td style="text-align: right">{meta_qtdeprodutos_ano}</td>
            <td style="text-align: right">{qtde_produtos_ano}</td> 
            <td style="text-align: center"><font color="{cor_qtde_ano}">◙</font></td>
        </tr>
        <tr>
            <td>Ticket Médio</td>
            <td style="text-align: right">R$ {meta_ticketmedia_ano:.2f}</td> 
            <td style="text-align: right">R$ {ticket_medio_ano:.2f}</td>
            <td style="text-align: center"><font color="{cor_ticket_ano}">◙</font></td>
        </tr>
    </table>
    <br>

    <p>Em anexo encontra-se a planilha com todas as movimentações caso necessite de mais detalhes.</p>

    <p>Para quaisquers dúvidas fico á disposição.</p>
    <p>Cordialmente</p>
    <p>Rafael Oliveira</p>

    
    '''
    
    

    #anexos 
    attachment = pathlib.Path.cwd()/ caminho_backup / loja / f'{dia_indicador.day}_{dia_indicador.month}_{loja}.xlsx'
    mail.Attachments.Add(str(attachment))

    #enviar
    mail.Send()
    print( f'E-mail da loja {loja} enviado com sucesso.')


E-mail da loja Iguatemi Esplanada enviado com sucesso.
E-mail da loja Shopping Midway Mall enviado com sucesso.
E-mail da loja Norte Shopping enviado com sucesso.
E-mail da loja Shopping Iguatemi Fortaleza enviado com sucesso.
E-mail da loja Shopping União de Osasco enviado com sucesso.
E-mail da loja Shopping Center Interlagos enviado com sucesso.
E-mail da loja Rio Mar Recife enviado com sucesso.
E-mail da loja Salvador Shopping enviado com sucesso.
E-mail da loja Rio Mar Shopping Fortaleza enviado com sucesso.
E-mail da loja Shopping Center Leste Aricanduva enviado com sucesso.
E-mail da loja Ribeirão Shopping enviado com sucesso.
E-mail da loja Shopping Morumbi enviado com sucesso.
E-mail da loja Parque Dom Pedro Shopping enviado com sucesso.
E-mail da loja Bourbon Shopping SP enviado com sucesso.
E-mail da loja Palladium Shopping Curitiba enviado com sucesso.
E-mail da loja Passei das Águas Shopping enviado com sucesso.
E-mail da loja Center Shopping Uberlândia enviado com sucesso

In [15]:
print(ticket_medio_dia)



nan


### Passo 5 - Criar ranking para diretoria

In [50]:
faturamento_lojas = vendas[['Loja','Valor Final']].groupby('Loja').sum()
faturamento_lojas_ano = faturamento_lojas.sort_values(by='Valor Final', ascending=False)

nome_arquivo = '{}_{}_Ranking Anual.xlsx'.format(dia_indicador.day, dia_indicador.month)
faturamento_lojas_ano.to_excel(r'Backup Arquivos Lojas\{}'.format(nome_arquivo))

vendas_dia = vendas.loc[vendas['Data']== dia_indicador, :]
faturamento_lojas_dia = vendas_dia[['Loja','Valor Final']].groupby('Loja').sum()
faturamento_lojas_dia = faturamento_lojas_dia.sort_values(by='Valor Final', ascending=False)

nome_arquivo = '{}_{}_Ranking Dia.xlsx'.format(dia_indicador.day,dia_indicador.month)
faturamento_lojas_dia.to_excel(r'Backup Arquivos Lojas\{}'.format(nome_arquivo))

### Passo 6 - Enviar e-mail para diretoria

In [51]:
#enviar e-mail
outlook = win32.Dispatch('outlook.application')

mail = outlook.CreateItem(0)
mail.To = emails.loc[emails['Loja']== 'Diretoria','E-mail'].values[0]
mail.Subject = f'Ranking Dia {dia_indicador.day}/{dia_indicador.month}'
mail.Body = f''' 
Prezados bom dia,

Apresentado abaixo o ranqueamento e resultados obtidos por Loja:

Melhor loja em faturamento do Dia: Loja {faturamento_lojas_dia.index[0]} com faturamento de R$ {faturamento_lojas_dia.iloc[0,0]:.2f}
Pior loja em faturamento do Dia: Loja {faturamento_lojas_dia.index[-1]} com faturamento de R$ {faturamento_lojas_dia.iloc[-1,0]:.2f}

Melhor loja em faturamento do Ano: Loja {faturamento_lojas_ano.index[0]} com faturamento de R$ {faturamento_lojas_ano.iloc[0,0]:.2f}
Pior loja em faturamento do Ano: Loja {faturamento_lojas_ano.index[-1]} com faturamento de R$ {faturamento_lojas_ano.iloc[-1,0]:.2f}

Para mais detalhes, consta em anexo os rankings completos do ano e do dia de todas as lojas.

Qualquer dúvida estou á disposição.

Cordialmente.
Rafael Oliveira

'''

#anexos
attachment = pathlib.Path.cwd() / caminho_backup / f'{dia_indicador.day}_{dia_indicador.month}_Ranking Anual.xlsx'
mail.Attachments.Add(str(attachment))
attachment = pathlib.Path.cwd() / caminho_backup / f'{dia_indicador.day}_{dia_indicador.month}_Ranking Dia.xlsx'
mail.Attachments.Add(str(attachment))

mail.Send()
print('E-mail da diretoria enviado com sucesso')


E-mail da diretoria enviado com sucesso
